In [129]:
import numpy as np
from PIL import Image
import random
np.random.seed(12)
random.seed(12)

In [130]:
def int_of_bytes(bytes):
    return int.from_bytes(bytes, "big")


def open_image(path):
    f = open(path, "rb")
    f.read(4)  # magic number
    num_images = int_of_bytes(f.read(4))
    rows = int_of_bytes(f.read(4))
    columns = int_of_bytes(f.read(4))
    f.close()
    data = np.fromfile(path, dtype="uint8")
    data = data[16:]
    return np.reshape(data, (num_images, rows, columns))


def display_image(np_array):
    im = Image.fromarray(np_array)
    im = im.convert("L")
    display(im)


def open_label(path):
    f = open(path, "rb")
    f.read(4)  # magic number
    num_images = int_of_bytes(f.read(4))
    f.close()
    data = np.fromfile(path, dtype="uint8")
    data = data[8:]
    return np.reshape(data, (num_images))


In [131]:
images_train = open_image("train-images-idx3-ubyte")
labels_train = open_label("train-labels-idx1-ubyte")
images_test = open_image("t10k-images-idx3-ubyte")
labels_test = open_label("t10k-labels-idx1-ubyte")

In [132]:
def p_sigmoid(x):
    y = sigmoid(x)
    return y * (1 - y)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def prepare_data(images, labels):
    return [(i.reshape(i.size, 1), label_to_vector(l)) for i, l in zip(images, labels)]


def label_to_vector(label: int):
    r = np.zeros((10, 1))
    r[label][0] = 1
    return r


def vector_to_label(vec):
    return np.argmax(vec)



In [133]:
class Network:
    def __init__(self, sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(x, 1) for x in sizes[1:]]
        self.weights = [np.random.randn(x, y) for x, y in zip(sizes[1:], sizes)]

    def sdg(self, training_data, epochs, mini_batch_size, eta, test_data):
        n = len(training_data)
        for i in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[x : x + mini_batch_size]
                for x in range(0, n, mini_batch_size)
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {i}: {self.evaluate(test_data)} / {len(test_data)}")
            else:
                print(f"Epoch {i} complete")

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [
            b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)
        ]
        self.weights = [
            w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)
        ]

    def backprop(self, x, y):
        zs = []
        activations = [x]
        activation = x
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            activation = sigmoid(z)
            zs.append(z)
            activations.append(activation)
        output_error = (activations[-1] - y) * p_sigmoid(zs[-1])
        nabla_b[-1] = output_error
        nabla_w[-1] = output_error @ activations[-2].transpose()

        for i in range(2, self.num_layers):
            output_error = (
                self.weights[-i + 1].transpose() @ output_error
            ) * p_sigmoid(zs[-i])
            nabla_b[-i] = output_error
            nabla_w[-i] = output_error @ activations[-i - 1].transpose()
        return nabla_b, nabla_w

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == vector_to_label(y)) for (x, y) in test_results)

    def feedforward(self, inp):
        for b, w in zip(self.biases, self.weights):
            inp = sigmoid(np.dot(w, inp) + b)
        return inp


In [134]:
network = Network([784, 30, 10])
def main():
    training_data = prepare_data(images_train, labels_train)
    test_data = prepare_data(images_test, labels_test)
    network.sdg(training_data, 30, 10, 3.0, training_data)


In [135]:
main()

/var/folders/k8/c8_zkt093fv9l1wmmxjpjt100000gn/T/ipykernel_46487/363644159.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0: 13774 / 60000
Epoch 1: 8728 / 60000
Epoch 2: 11715 / 60000
Epoch 3: 9487 / 60000


In [ ]:
def predict_label(image):
    vl = network.feedforward(image.reshape(image.size, 1))
    return vector_to_label(vl)

In [ ]:
display_image(images_test[0])